# Using Machine Learning to Forecast Air Quality in Beijing

## 1 - Data Wrangling

### Import Python Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from collections import Counter

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


### Load data set

In [2]:
# Data obtained from UCI Repository (https://archive.ics.uci.edu/ml/datasets/PM2.5+Data+of+Five+Chinese+Cities)
df = pd.read_csv('data/BeijingPM20100101_20151231.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   No               52584 non-null  int64  
 1   year             52584 non-null  int64  
 2   month            52584 non-null  int64  
 3   day              52584 non-null  int64  
 4   hour             52584 non-null  int64  
 5   season           52584 non-null  int64  
 6   PM_Dongsi        25052 non-null  float64
 7   PM_Dongsihuan    20508 non-null  float64
 8   PM_Nongzhanguan  24931 non-null  float64
 9   PM_US Post       50387 non-null  float64
 10  DEWP             52579 non-null  float64
 11  HUMI             52245 non-null  float64
 12  PRES             52245 non-null  float64
 13  TEMP             52579 non-null  float64
 14  cbwd             52579 non-null  object 
 15  Iws              52579 non-null  float64
 16  precipitation    52100 non-null  float64
 17  Iprec       

,No,year,month,day,hour,season,PM_Dongsi,PM_Dongsihuan,PM_Nongzhanguan,PM_US Post,DEWP,HUMI,PRES,TEMP,cbwd,Iws,precipitation,Iprec
0,1,2010,1,1,0,4,NaN,NaN,NaN,NaN,-21.0,43.0,1021.0,-11.0,NW,1.79,0.0,0.0
1,2,2010,1,1,1,4,NaN,NaN,NaN,NaN,-21.0,47.0,1020.0,-12.0,NW,4.92,0.0,0.0
2,3,2010,1,1,2,4,NaN,NaN,NaN,NaN,-21.0,43.0,1019.0,-11.0,NW,6.71,0.0,0.0
3,4,2010,1,1,3,4,NaN,NaN,NaN,NaN,-21.0,55.0,1019.0,-14.0,NW,9.84,0.0,0.0
4,5,2010,1,1,4,4,NaN,NaN,NaN,NaN,-20.0,51.0,1018.0,-12.0,NW,12.97,0.0,0.0


In [3]:
print (df.loc[df["PM_US Post"].first_valid_index()])

No                   24
year               2010
month                 1
day                   1
hour                 23
season                4
PM_Dongsi           NaN
PM_Dongsihuan       NaN
PM_Nongzhanguan     NaN
PM_US Post          129
DEWP                -17
HUMI                 41
PRES               1020
TEMP                 -5
cbwd                 cv
Iws                0.89
precipitation         0
Iprec                 0
Name: 23, dtype: object


In [4]:
df = df.iloc[24:, :]

In [5]:
df.interpolate(inplace=True)

### Re-index DataFrame and clean up column labels

In [6]:
# Index dataframe using datetime
df.index = pd.to_datetime(df.year*1000000 + df.month*10000 + df.day*100 + df.hour, format='%Y%m%d%H')

In [7]:
df.drop(["No", "day", "hour", "PM_Dongsi", "PM_Dongsihuan", "PM_Nongzhanguan"], axis=1, inplace=True)

In [8]:
# Rename columns to be easily recognizable
df = df.rename(columns={'PM_US Post': 'pm25',
                        'DEWP': 'dew_point',
                        'HUMI': 'humidity',
                        'PRES': 'pressure',
                        'TEMP' : 'temp',
                        'cbwd': 'wind_dir',
                        'Iws': 'wind_speed',
                        'Iprec': 'cum_precipitation'})

In [9]:
def wind_dir_mode(x):
    c = Counter(x)
    return(c.most_common()[0][0])

In [10]:
df = df.resample('D').wind_dir.agg(wind_dir=wind_dir_mode).join(df.drop("cum_precipitation", axis=1).resample('D').mean()).join(df.cum_precipitation.resample("D").max())

In [11]:
df.head()

,wind_dir,year,month,season,pm25,dew_point,humidity,pressure,temp,wind_speed,precipitation,cum_precipitation
2010-01-02,SE,2010,1,4,144.333333,-8.500000,77.937500,1024.750000,-5.125000,24.860000,0.000000,0.0
2010-01-03,SE,2010,1,4,78.375000,-10.125000,87.916667,1022.791667,-8.541667,70.937917,0.466667,11.2
2010-01-04,NW,2010,1,4,29.291667,-20.875000,46.208333,1029.291667,-11.500000,111.160833,0.000000,0.0
2010-01-05,NW,2010,1,4,43.541667,-24.583333,42.041667,1033.625000,-14.458333,56.920000,0.000000,0.0
2010-01-06,NE,2010,1,4,59.375000,-23.708333,39.208333,1033.750000,-12.541667,18.511667,0.000000,0.0


In [12]:
df.to_csv("data/dailypm25.csv")